In [ ]:
import QuantumRingsLib
print (QuantumRingsLib.__version__)

In [ ]:
import QuantumRingsLib
from QuantumRingsLib import QuantumRegister, ClassicalRegister, QuantumCircuit
from QuantumRingsLib import QuantumRingsProvider, job_monitor
import math
import numpy as np
from matplotlib import pyplot as plt

# Initialize the provider and backend
provider = QuantumRingsProvider(
    token='rings-64.8fourH2rnXVCbZ3QCI0wFKibkUij6WHS',
    name='abku2504@gmail.com'
)
backend = provider.get_backend("scarlet_quantum_rings")
shots = 1024
provider.active_account()

def iqft_cct(qc, reg, n):
    """Applies the inverse QFT on a register of n qubits."""
    for i in range(n):
        for j in range(1, i+1):
            qc.cu1(-math.pi / 2**(i - j + 1), reg[j - 1], reg[i])
        qc.h(reg[i])
    qc.barrier()
    return

def plot_histogram(counts, title=""):
    """Plots the histogram of measurement counts."""
    fig, ax = plt.subplots(figsize=(10, 7))
    plt.xlabel("States")
    plt.ylabel("Counts")
    mylist = [key for key, val in counts.items() for _ in range(val)]
    unique, inverse = np.unique(mylist, return_inverse=True)
    bin_counts = np.bincount(inverse)
    plt.bar(unique, bin_counts)
    maxFreq = max(counts.values())
    plt.ylim(ymax=np.ceil(maxFreq / 10) * 10 if maxFreq % 10 else maxFreq + 10)
    plt.title(title)
    plt.show()

def modular_exponentiation(qc, exp_reg, target_reg, a, N):
    """
    Placeholder: Implements the modular exponentiation circuit which computes:
        |x>|1> -> |x>|a^x mod N>
    In a full implementation, this would be built using repeated controlled multiplications.
    """
    # TODO: Implement the full modular exponentiation circuit.
    # For demonstration purposes, this is left as a placeholder.
    qc.barrier()
    return

def build_shor_circuit(N, a, n_exp, n_target):
    """
    Builds a Shor's algorithm circuit for factoring N with chosen base a.
    n_exp: number of qubits in the exponent register.
    n_target: number of qubits in the target register.
    """
    total_qubits = n_exp + n_target
    q = QuantumRegister(total_qubits, 'q')
    c = ClassicalRegister(n_exp, 'c')
    qc = QuantumCircuit(q, c)
    
    # Step 1: Prepare the exponent register in superposition
    for i in range(n_exp):
        qc.h(q[i])
    
    # Step 2: Initialize the target register to |1>
    # (Assuming the target register starts in |0>, apply an X gate to set it to |1>.)
    qc.x(q[n_exp])
    qc.barrier()
    
    # Step 3: Modular Exponentiation: |x>|1> -> |x>|a^x mod N>
    modular_exponentiation(qc, q[:n_exp], q[n_exp:], a, N)
    qc.barrier()
    
    # Step 4: Apply the inverse QFT on the exponent register
    iqft_cct(qc, q[:n_exp], n_exp)
    
    # Step 5: Measure the exponent register
    for i in range(n_exp):
        qc.measure(q[i], c[i])
    
    return qc

def process_counts_to_get_period(counts):
    """
    Placeholder: Process measurement counts (using continued fractions) to extract the period r.
    A full implementation would analyze the histogram, convert the observed state to a phase,
    and then compute r via continued fraction expansion.
    """
    # For demonstration, return a dummy period value.
    return 4  # Placeholder

def factorize(N, a, n_exp, n_target):
    """Runs the quantum circuit for factoring N and extracts the factors."""
    qc = build_shor_circuit(N, a, n_exp, n_target)
    qc.draw('mpl')  # Visualize the circuit
    job = backend.run(qc, shots=shots)
    job_monitor(job)
    result = job.result()
    counts = result.get_counts()
    print("Measurement counts:", counts)
    plot_histogram(counts, title=f"Results for N={N}")
    
    # Post-processing: extract the period r from measurement counts
    r = process_counts_to_get_period(counts)
    print(f"Extracted period r = {r}")
    
    # Compute factors using the standard method from Shor's algorithm
    factor1 = math.gcd(pow(a, r//2) - 1, N)
    factor2 = math.gcd(pow(a, r//2) + 1, N)
    print(f"Factors of {N}: {factor1} and {factor2}")
    return factor1, factor2

# List of semiprime numbers (bit size : semiprime)
semiprimes = {
    8: 143
}

# Loop over each semiprime and attempt factorization
for bits, N in semiprimes.items():
    # Choose a base 'a' that is coprime with N. (For demonstration, try a=7; adjust if needed.)
    a = 7
    if math.gcd(a, N) != 1:
        a = 2  # or another small prime
    
    # Determine the number of qubits:
    # - Exponent register: typically around 2*ceil(log2(N))
    # - Target register: ceil(log2(N))
    n_target = math.ceil(math.log(N, 2))
    n_exp = 2 * n_target
    print(f"\nFactoring N = {N} (bit size {bits}) using base a = {a}")
    factorize(N, a, n_exp, n_target)

# TESTING

In [ ]:
import math
import numpy as np
from matplotlib import pyplot as plt

# Import UnitaryGate from qiskit.circuit.library (compatible with Qiskit 1.3.2)
from qiskit.circuit.library import UnitaryGate
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister

# Use the new provider details
from QuantumRingsLib import QuantumRingsProvider, job_monitor

# New provider with updated token and account name
provider = QuantumRingsProvider(
    token='rings-200.b8X795Phx09UGpAHt8noxrzbDiKdFL1h',
    name='abku2504@gmail.com'
)
# Get the backend (scarlet_quantum_rings remains the same)
backend = provider.get_backend("scarlet_quantum_rings")

# Experiment with fewer shots for faster execution
shots = 256

def iqft_cct(qc, reg, n):
    """Applies the inverse QFT on register 'reg' (length n)."""
    for i in range(n):
        for j in range(1, i + 1):
            qc.cu1(-math.pi / 2**(i - j + 1), reg[j - 1], reg[i])
        qc.h(reg[i])
    qc.barrier()

def plot_histogram(counts, title=""):
    """Plots measurement counts."""
    states = list(counts.keys())
    freqs = list(counts.values())
    plt.figure(figsize=(10, 7))
    plt.bar(states, freqs)
    plt.xlabel("States")
    plt.ylabel("Counts")
    plt.title(title)
    plt.show()

def modular_multiplication_gate(multiplier, N, n_target):
    """
    Constructs a unitary gate U that maps |y> to |(multiplier * y) mod N>
    for y < N and leaves states with y >= N unchanged.
    """
    dim = 2**n_target
    U = np.zeros((dim, dim), dtype=complex)
    for y in range(dim):
        if y < N:
            new_y = (multiplier * y) % N
            U[new_y, y] = 1.0
        else:
            U[y, y] = 1.0
    label = f"mult_by_{multiplier}_mod_{N}"
    return UnitaryGate(U, label=label)

def controlled_modular_multiplication(qc, control, target, multiplier, N):
    """
    Appends a controlled modular multiplication gate to qc.
    The gate multiplies the target register by 'multiplier' modulo N,
    controlled by the qubit 'control'.
    """
    n_target = len(target)
    U_gate = modular_multiplication_gate(multiplier, N, n_target)
    controlled_U = U_gate.control(1)
    qc.append(controlled_U, [control] + target)

def build_shor_circuit(N, a, n_exp, n_target):
    """
    Builds a Shor circuit for factoring N using base a.
    
    Args:
      N: The integer to factor.
      a: The chosen base (must be coprime with N).
      n_exp: Number of qubits in the exponent (control) register.
      n_target: Number of qubits in the target register.
      
    Returns:
      A QuantumCircuit implementing the algorithm.
    """
    # Create separate registers for the exponent and target
    exp = QuantumRegister(n_exp, 'exp')
    target = QuantumRegister(n_target, 'target')
    c = ClassicalRegister(n_exp, 'c')
    qc = QuantumCircuit(exp, target, c)
    
    # Step 1: Superposition on the exponent register
    qc.h(exp)
    
    # Step 2: Initialize the target register to |1>
    qc.x(target[0])
    qc.barrier()
    
    # Step 3: Apply controlled modular multiplication for each exponent qubit.
    # For testing, we use a reduced exponent register (n_exp set manually).
    for j in range(n_exp):
        multiplier = pow(a, 2**j, N)
        controlled_modular_multiplication(qc, exp[j], list(target), multiplier, N)
        qc.barrier()
    
    # Step 4: Inverse QFT on the exponent register
    iqft_cct(qc, exp, n_exp)
    
    # Step 5: Measure the exponent register
    qc.measure(exp, c)
    
    return qc

def process_counts_to_get_period(counts):
    """
    Dummy post-processing: in a full implementation, one would apply a continued fractions
    algorithm to the measurement outcomes to extract the period r.
    Here we return a dummy value for demonstration.
    """
    return 4  # Dummy value

def factorize(N, a, n_exp, n_target):
    """
    Runs the Shor circuit for factoring N and prints the factors.
    """
    qc = build_shor_circuit(N, a, n_exp, n_target)
    qc.draw('mpl')  # Visualize the circuit
    job = backend.run(qc, shots=shots)
    job_monitor(job)
    result = job.result()
    counts = result.get_counts()
    print("Measurement counts:", counts)
    plot_histogram(counts, title=f"Results for N={N}")
    
    # Dummy period extraction (to be replaced by proper continued fractions post-processing)
    r = process_counts_to_get_period(counts)
    print(f"Extracted period r = {r}")
    
    # Compute factors provided r is even and valid
    factor1 = math.gcd(pow(a, r // 2) - 1, N)
    factor2 = math.gcd(pow(a, r // 2) + 1, N)
    print(f"Factors of {N}: {factor1} and {factor2}")
    return factor1, factor2

# --- Experiment: Factor 143 with adjusted parameters ---
# For N = 143, we set:
# n_target = ceil(log₂(143)) must be at least 8.
# Instead of n_exp = 16, we choose n_exp = 6 for a faster run.
N = 143
a = 2  # Trying base = 2 instead of 7
n_target = math.ceil(math.log(N, 2))
n_exp = 6  # Reduced exponent register for faster simulation

print(f"Factoring N = {N} using base a = {a}")
factorize(N, a, n_exp, n_target)